In [1]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd

In [22]:
class PlacesClient:
    def __init__(self, token):
        self.base_url = 'https://data.cdc.gov/api/v3/views/'
        self.session = requests.Session()
        self.session.headers.update({
            'X-App-Token': token
        })

    def _make_request(self, url, params=None):
        """
        Make a get request to the API and return responses in JSON
        """
        try:
            response = self.session.get(url, params=params)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.HTTPError as e:
            print(f"API Error: {e}")
            raise
    
    def _json_to_df(self, data):
        """
        Transform JSON data into pandas DataFrame.
        """
        df = pd.DataFrame(data)
        # remove the API's metadata
        df = df.drop([':id', ':version', ':created_at', ':updated_at'], axis=1, errors='ignore')
        # convert numeric variables
        numeric_cols = ['data_value', 'low_confidence_limit', 'high_confidence_limit', 'totalpopulation']
        for col in numeric_cols:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col])
        return df

    def get_county_data(self, release='2025'):
        """
        Retrieve county-level health-risk behaviors and health outcomes data from The CDC PLACES API.
        
        Parameters
        ----------
        release : string
            The version of release to retrieve from.

        Returns
        -------
        county_df : pandas DataFrame
            A dataframe containing information of county-level PLACES data
        
        Examples
        --------
        >>> df = get_county_data('2023')
        >>> df.head()
        """
        release_ids = {
            '2025': 'swc5-untb',
            '2024': 'fu4u-a9bh',
            '2023': 'h3ej-a9ec',
            '2022': 'duw2-7jbt',
            '2021': 'pqpp-u99h'
        }
        
        if not isinstance(release, str):
            raise TypeError("The release must be a string.")
        elif release not in release_ids:
            raise ValueError("This release version is not supported.")
        else:
            url = self.base_url + release_ids[release] + '/query.json'

        data = self._make_request(url)
        county_df = self._json_to_df(data)
        
        # Only keep measures categorized as health outcomes and health risk behaviors
        county_df = county_df[county_df['category'].isin(['Health Outcomes', 'Health Risk Behaviors'])]
        county_df = county_df.reset_index(drop=True)
        return county_df


In [23]:
load_dotenv()
token = os.getenv('CDC_API_TOKEN')

client = PlacesClient(token=token)
county_data_2023 = client.get_county_data('2023')
county_data_2023

,year,stateabbr,statedesc,locationname,datasource,category,measure,data_value_unit,data_value_type,data_value,low_confidence_limit,high_confidence_limit,totalpopulation,locationid,categoryid,measureid,datavaluetypeid,short_question_text,geolocation
0,2021,TX,Texas,Potter,BRFSS,Health Outcomes,Current asthma among adults aged >=18 years,%,Crude prevalence,10.1,8.9,11.5,116547,48375,HLTHOUT,CASTHMA,CrdPrv,Current Asthma,"{'type': 'Point', 'coordinates': [-101.893804,..."
1,2021,TN,Tennessee,Trousdale,BRFSS,Health Outcomes,Arthritis among adults aged >=18 years,%,Crude prevalence,25.4,21.8,29.1,12035,47169,HLTHOUT,ARTHRITIS,CrdPrv,Arthritis,"{'type': 'Point', 'coordinates': [-86.1566909,..."
2,2021,TX,Texas,Shackelford,BRFSS,Health Outcomes,Arthritis among adults aged >=18 years,%,Crude prevalence,29.0,26.5,31.7,3212,48417,HLTHOUT,ARTHRITIS,CrdPrv,Arthritis,"{'type': 'Point', 'coordinates': [-99.3470045,..."
3,2020,UT,Utah,Summit,BRFSS,Health Outcomes,All teeth lost among adults aged >=65 years,%,Crude prevalence,6.3,3.5,9.9,43093,49043,HLTHOUT,TEETHLOST,CrdPrv,All Teeth Lost,"{'type': 'Point', 'coordinates': [-110.9684862..."
4,2021,VA,Virginia,Amelia,BRFSS,Health Outcomes,Stroke among adults aged >=18 years,%,Crude prevalence,4.2,3.7,4.8,13268,51007,HLTHOUT,STROKE,CrdPrv,Stroke,"{'type': 'Point', 'coordinates': [-77.9774619,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104881,2021,RI,Rhode Island,Bristol,BRFSS,Health Outcomes,Current asthma among adults aged >=18 years,%,Crude prevalence,10.7,9.4,12.2,50818,44001,HLTHOUT,CASTHMA,CrdPrv,Current Asthma,"{'type': 'Point', 'coordinates': [-71.2866874,..."
104882,2021,TX,Texas,Brown,BRFSS,Health Outcomes,High cholesterol among adults aged >=18 years ...,%,Crude prevalence,40.4,35.3,45.5,38192,48049,HLTHOUT,HIGHCHOL,CrdPrv,High Cholesterol,"{'type': 'Point', 'coordinates': [-98.998456, ..."
104883,2021,TX,Texas,Grayson,BRFSS,Health Outcomes,Chronic kidney disease among adults aged >=18 ...,%,Age-adjusted prevalence,2.8,2.6,3.2,139336,48181,HLTHOUT,KIDNEY,AgeAdjPrv,Chronic Kidney Disease,"{'type': 'Point', 'coordinates': [-96.675693, ..."
104884,2021,SC,South Carolina,Richland,BRFSS,Health Risk Behaviors,No leisure-time physical activity among adults...,%,Crude prevalence,23.9,20.3,27.5,418307,45079,RISKBEH,LPA,CrdPrv,Physical Inactivity,"{'type': 'Point', 'coordinates': [-80.8980371,..."


In [24]:
county_data_2023.tail()

,year,stateabbr,statedesc,locationname,datasource,category,measure,data_value_unit,data_value_type,data_value,low_confidence_limit,high_confidence_limit,totalpopulation,locationid,categoryid,measureid,datavaluetypeid,short_question_text,geolocation
104881,2021,RI,Rhode Island,Bristol,BRFSS,Health Outcomes,Current asthma among adults aged >=18 years,%,Crude prevalence,10.7,9.4,12.2,50818,44001,HLTHOUT,CASTHMA,CrdPrv,Current Asthma,"{'type': 'Point', 'coordinates': [-71.2866874,..."
104882,2021,TX,Texas,Brown,BRFSS,Health Outcomes,High cholesterol among adults aged >=18 years ...,%,Crude prevalence,40.4,35.3,45.5,38192,48049,HLTHOUT,HIGHCHOL,CrdPrv,High Cholesterol,"{'type': 'Point', 'coordinates': [-98.998456, ..."
104883,2021,TX,Texas,Grayson,BRFSS,Health Outcomes,Chronic kidney disease among adults aged >=18 ...,%,Age-adjusted prevalence,2.8,2.6,3.2,139336,48181,HLTHOUT,KIDNEY,AgeAdjPrv,Chronic Kidney Disease,"{'type': 'Point', 'coordinates': [-96.675693, ..."
104884,2021,SC,South Carolina,Richland,BRFSS,Health Risk Behaviors,No leisure-time physical activity among adults...,%,Crude prevalence,23.9,20.3,27.5,418307,45079,RISKBEH,LPA,CrdPrv,Physical Inactivity,"{'type': 'Point', 'coordinates': [-80.8980371,..."
104885,2020,WI,Wisconsin,La Crosse,BRFSS,Health Outcomes,All teeth lost among adults aged >=65 years,%,Crude prevalence,8.1,5.0,11.8,120433,55063,HLTHOUT,TEETHLOST,CrdPrv,All Teeth Lost,"{'type': 'Point', 'coordinates': [-91.1117584,..."


In [83]:
def get_measure_by_state(df, measure = None, state = None):
    if measure == None or state == None:
        return "Missing parameters."
    else:
        result = df[(df['measureid'] == measure) & (df['stateabbr'] == state)]
        return result

In [84]:
get_measure_by_state(county_data_2023, measure='CASTHMA', state='TX')

,year,stateabbr,statedesc,locationname,datasource,category,measure,data_value_unit,data_value_type,data_value,low_confidence_limit,high_confidence_limit,totalpopulation,locationid,categoryid,measureid,datavaluetypeid,short_question_text,geolocation
0,2021,TX,Texas,Potter,BRFSS,Health Outcomes,Current asthma among adults aged >=18 years,%,Crude prevalence,10.1,8.9,11.5,116547,48375,HLTHOUT,CASTHMA,CrdPrv,Current Asthma,"{'type': 'Point', 'coordinates': [-101.893804,..."
164750,2021,TX,Texas,Tarrant,BRFSS,Health Outcomes,Current asthma among adults aged >=18 years,%,Crude prevalence,9.7,8.6,10.9,2126477,48439,HLTHOUT,CASTHMA,CrdPrv,Current Asthma,"{'type': 'Point', 'coordinates': [-97.2912241,..."
164900,2021,TX,Texas,Montgomery,BRFSS,Health Outcomes,Current asthma among adults aged >=18 years,%,Age-adjusted prevalence,9.3,8.1,10.6,648886,48339,HLTHOUT,CASTHMA,AgeAdjPrv,Current Asthma,"{'type': 'Point', 'coordinates': [-95.50295, 3..."
165287,2021,TX,Texas,Hall,BRFSS,Health Outcomes,Current asthma among adults aged >=18 years,%,Crude prevalence,9.8,8.6,11.0,2845,48191,HLTHOUT,CASTHMA,CrdPrv,Current Asthma,"{'type': 'Point', 'coordinates': [-100.5763434..."
165388,2021,TX,Texas,Brewster,BRFSS,Health Outcomes,Current asthma among adults aged >=18 years,%,Age-adjusted prevalence,8.8,7.7,10.1,9450,48043,HLTHOUT,CASTHMA,AgeAdjPrv,Current Asthma,"{'type': 'Point', 'coordinates': [-103.2524579..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228031,2021,TX,Texas,Delta,BRFSS,Health Outcomes,Current asthma among adults aged >=18 years,%,Age-adjusted prevalence,10.2,8.9,11.5,5392,48119,HLTHOUT,CASTHMA,AgeAdjPrv,Current Asthma,"{'type': 'Point', 'coordinates': [-95.6733504,..."
228160,2021,TX,Texas,McLennan,BRFSS,Health Outcomes,Current asthma among adults aged >=18 years,%,Crude prevalence,10.1,8.9,11.5,263115,48309,HLTHOUT,CASTHMA,CrdPrv,Current Asthma,"{'type': 'Point', 'coordinates': [-97.2015108,..."
228215,2021,TX,Texas,Ward,BRFSS,Health Outcomes,Current asthma among adults aged >=18 years,%,Crude prevalence,9.6,8.4,10.8,11194,48475,HLTHOUT,CASTHMA,CrdPrv,Current Asthma,"{'type': 'Point', 'coordinates': [-103.1051127..."
228512,2021,TX,Texas,Swisher,BRFSS,Health Outcomes,Current asthma among adults aged >=18 years,%,Crude prevalence,10.1,8.8,11.4,7008,48437,HLTHOUT,CASTHMA,CrdPrv,Current Asthma,"{'type': 'Point', 'coordinates': [-101.7438647..."
